In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../..")

In [ ]:
import os
import analytiq_data as ad
import asyncio
import litellm

In [ ]:
# Set the MONGODB_URI environment variable
os.environ["MONGODB_URI"] = "mongodb://localhost:27017"

In [ ]:
litellm.model_list

In [ ]:
litellm.model_cost["j2-light"]

In [ ]:
litellm.get_model_info(model="groq/deepseek-r1-distill-llama-70b")

In [ ]:
litellm.models_by_provider

In [ ]:
from litellm import get_supported_openai_params

response = get_supported_openai_params(model="bedrock/anthropic.claude-3", custom_llm_provider="bedrock")

print(response) # ["max_tokens", "tools", "tool_choice", "stream"]

In [22]:
def get_embedding_models(provider="openai"):
    """
    Get all embedding models for a given provider.
    
    Args:
        provider: The litellm provider name (e.g., "openai", "cohere", "azure")
    
    Returns:
        List of dictionaries, each containing:
        - name: Model name
        - dimensions: Embedding vector dimensions (output_vector_size)
        - output_cost_per_token: Cost per token for output (typically 0 for embeddings)
        - output_cost_per_token_batches: Cost per token for batched output (if available)
    """
    models = litellm.models_by_provider.get(provider, [])
    embedding_models = []
    
    for model in models:
        try:
            model_info = litellm.get_model_info(model)
            # Check if this is an embedding model
            if model_info.get('mode') == 'embedding':
                embedding_models.append({
                    'name': model,
                    'dimensions': model_info.get('output_vector_size'),
                    'output_cost_per_token': model_info.get('output_cost_per_token'),
                    'output_cost_per_token_batches': model_info.get('output_cost_per_token_batches')
                })
        except Exception as e:
            # Skip models that can't be queried
            pass
    
    return embedding_models

# Example usage
embedding_models = get_embedding_models("openai")
print("OpenAI Embedding Models:")
for model in embedding_models:
    print(f"\n  Model: {model['name']}")
    print(f"    Dimensions: {model['dimensions']}")
    print(f"    Output cost per token: {model['output_cost_per_token']}")
    print(f"    Output cost per token (batches): {model['output_cost_per_token_batches']}")

OpenAI Embedding Models:

  Model: text-embedding-ada-002
    Dimensions: 1536
    Output cost per token: 0.0
    Output cost per token (batches): None

  Model: text-embedding-3-small
    Dimensions: 1536
    Output cost per token: 0.0
    Output cost per token (batches): 0.0

  Model: text-embedding-3-large
    Dimensions: 3072
    Output cost per token: 0.0
    Output cost per token (batches): 0.0

  Model: text-embedding-ada-002-v2
    Dimensions: None
    Output cost per token: 0.0
    Output cost per token (batches): 0.0


In [18]:
# Get all OpenAI models
openai_models = litellm.models_by_provider.get("openai", [])

# Check each model's info to see if it supports embeddings
embedding_models = []
for model in openai_models:
    try:
        model_info = litellm.get_model_info(model)
        # Check if model supports embeddings (you may need to test or check litellm source)
        # For OpenAI, embedding models typically have "text-embedding" in the name
        if "embedding" in model.lower() or model_info.get("supports_embeddings", False):
            embedding_models.append(model)
    except Exception as e:
        # Skip models that can't be queried
        pass

print("OpenAI Embedding Models:")
for model in embedding_models:
    print(f"  - {model}")

OpenAI Embedding Models:
  - text-embedding-ada-002
  - text-embedding-3-small
  - text-embedding-3-large
  - text-embedding-ada-002-v2


In [ ]:
# Get the model card for text-embedding-3-small
#model_card = litellm.get_model_info("gpt-4o-mini")
model_card = litellm.get_model_info("text-embedding-3-large")
model_card


{'key': 'gpt-4o-mini',
 'max_tokens': 16384,
 'max_input_tokens': 128000,
 'max_output_tokens': 16384,
 'input_cost_per_token': 1.5e-07,
 'input_cost_per_token_flex': None,
 'input_cost_per_token_priority': 2.5e-07,
 'cache_creation_input_token_cost': None,
 'cache_creation_input_token_cost_above_200k_tokens': None,
 'cache_read_input_token_cost': 7.5e-08,
 'cache_read_input_token_cost_above_200k_tokens': None,
 'cache_read_input_token_cost_flex': None,
 'cache_read_input_token_cost_priority': 1.25e-07,
 'cache_creation_input_token_cost_above_1hr': None,
 'input_cost_per_character': None,
 'input_cost_per_token_above_128k_tokens': None,
 'input_cost_per_token_above_200k_tokens': None,
 'input_cost_per_query': None,
 'input_cost_per_second': None,
 'input_cost_per_audio_token': None,
 'input_cost_per_image_token': None,
 'input_cost_per_image': None,
 'input_cost_per_audio_per_second': None,
 'input_cost_per_video_per_second': None,
 'input_cost_per_token_batches': 7.5e-08,
 'output_cos

In [16]:
def list_embedding_models(provider="openai"):
    """List all embedding models for a given provider"""
    models = litellm.models_by_provider.get(provider, [])
    embedding_models = []
    
    for model in models:
        try:
            model_info = litellm.get_model_info(model)
            if model_info.get('mode') == 'embedding':
                embedding_models.append({
                    'model': model,
                    'dimensions': model_info.get('output_vector_size'),
                    'cost_per_token': model_info.get('input_cost_per_token'),
                    'max_tokens': model_info.get('max_input_tokens')
                })
        except:
            pass
    
    return embedding_models

# Usage
embedding_models = list_embedding_models("openai")
for model_info in embedding_models:
    print(f"{model_info['model']}: {model_info['dimensions']} dimensions, ${model_info['cost_per_token']} per token")

text-embedding-ada-002: 1536 dimensions, $1e-07 per token
text-embedding-3-small: 1536 dimensions, $2e-08 per token
text-embedding-3-large: 3072 dimensions, $1.3e-07 per token
text-embedding-ada-002-v2: None dimensions, $1e-07 per token
